In [10]:
import pandas as pd
import numpy as np
import datetime
import re
from datetime import timedelta
from datetime import date

### Carga de librerias y archivos

In [11]:
# Ruta al archivo Excel
archivo_excel = "C:/Users/contr/OneDrive/Desarrollo Diario/BI/Facturacion historica.xlsx"

# Nombre de la hoja
nombre_hoja = 'Repositorio FST'

# Lee el archivo de Excel en un DataFrame de pandas
df = pd.read_excel(archivo_excel, sheet_name=nombre_hoja)
df_= df.copy()
df.shape

(565894, 38)

### PASOS PARA LA NORMALIZACION DE DATA

In [12]:
nuevos_nombres = ["oficina","tipo_orden_trabajo","numero_orden_trabajo","tipo_trabajo","numero_documento_venta","fecha_venta","numero_siniestro","numero_orden","numero_correlacion","codigo_recepcionista","agente_confirmo_7d","fecha_apertura_orden_trabajo","fecha_termino_orden_trabajo","detalle_factura","rut_cliente","nombre_cliente","patente","marca","modelo","total_costo","total_neto_mano_obra","total_neto_repuestos","total_neto_accesorios","total_neto_lubricantes","total_neto_materiales","total_neto_otros","total_neto_documento","total_bruto_documento","cantidad_litros_lubricantes","forma_pago","vin","tipo_venta","tipo_mantenimiento","kilometros_vehiculo","posee_mpp","id_agente","numero_presupuesto", "destinatario_kovcas"]
df.columns = nuevos_nombres 
#Se crea columna contiene obd y se pasa a mayusculas lo que es detalle_factura
df= df.assign(total_ingresos = ((df['total_neto_accesorios'] + df['total_neto_lubricantes'] + df['total_neto_materiales'] + df['total_neto_repuestos']) - df['total_costo']) + df['total_neto_mano_obra'])
df['total_facturacion'] = df['total_neto_accesorios'] + df['total_neto_lubricantes'] + df['total_neto_materiales'] + df['total_neto_repuestos'] + df['total_neto_mano_obra']
df['vin'] = df['vin'].str.replace('O', '0')
df['año_venta'] = pd.DatetimeIndex(df['fecha_venta']).year
df['mes_venta'] = pd.DatetimeIndex(df['fecha_venta']).month
df['detalle_factura'].fillna('N/A', inplace=True)

df['contiene_obd'] = np.where(df['detalle_factura'].str.contains("OBD|ANTICLO", na=False), 'SI', 'NO')
#Se Agrega La Columna Nota Credito
df['nota_credito'] = np.where(df['total_neto_repuestos'] >= 0, 'SIN NC', 'CON NC')
df['oficina'] = df['oficina'].str.upper()
def asignar_unidad_negocios(oficina):
    if "NISSAN" in oficina:
        return "NISSAN"
    elif "TOYOTA" in oficina:
        return "TOYOTA"
    elif "DYP" in oficina:
        return "DYP"
    else:
        return "CHEVROLET"

df["unidad_negocios"] = df["oficina"].apply(asignar_unidad_negocios)
#Agregar columna tipo cliente
df['tipo_cliente'] = df['detalle_factura'].apply(lambda x: 'SEGURO' if 'SEGURO' in str(x) else 'GENERAL')
#Cambiar tipificacion de acuerdo a condicion
df.loc[df['posee_mpp'] == 'SI', 'tipo_trabajo'] = 'Mant. Prev.'
# Se re tipifica la columna Tipo Trabajo
df['tipo_trabajo'] = df['tipo_trabajo'].replace(["Acc. u otros", "Accesorios", "Corr.,Acc. u otros"], "CORRECTIVA")
df['tipo_trabajo'] = df['tipo_trabajo'].replace(["Mant. Prev."], "PREVENTIVA")
df['tipo_trabajo'] = df['tipo_trabajo'].replace(["CRÉDITO"], "CREDITO")
df['tipo_trabajo'] = df['tipo_trabajo'].replace(["GARANTÍA"], "GARANTIA")
# Se reemplan todos los Valores en la columna tiene_mpp de SI a NO en caso que el registro tenga MPP 
df.loc[df['contiene_obd'] == 'SI', 'posee_mpp'] = 'NO' 
# Se reemplan todos los Valores en la columna tiene_mpp de SI a NO en caso que el registro tenga MPP 
# df.loc[df['contiene_obd'] == 'SI', 'posee_mpp'] = 'NO' 

# Se reemplan todos los Valores en la columna tiene_mpp de SI a NO en caso que el registro tenga MPP 
df.loc[df['contiene_obd'] == 'SI', 'tipo_trabajo'] = 'CORRECTIVA' 
#Se normaliza la data en la columna *Tipo Trabajo*
df['tipo_trabajo'] = df['tipo_trabajo'].str.replace('Á', 'A').str.replace('É', 'E').str.replace('Í', 'I').str.replace('Ó', 'O').str.replace('Ú', 'U').str.upper()
df['tipo_trabajo'] = df['tipo_trabajo'].replace(np.nan, 'sin_tipo_trabajo', regex=True)
#Se normaliza la data en la columna *Tipo Trabajo*
df['tipo_orden_trabajo'] = df['tipo_orden_trabajo'].str.replace('Á', 'A').str.replace('É', 'E').str.replace('Í', 'I').str.replace('Ó', 'O').str.replace('Ú', 'U').str.upper()
df['tipo_orden_trabajo'] = df['tipo_orden_trabajo'].replace(np.nan, 'sin_orden_trabajo', regex=True)
# Se normaliza la data en la columna Tipo Orden Trabajo
df['tipo_orden_trabajo'].fillna('N/A', inplace=True)
df['tipo_orden_trabajo'] = df['tipo_orden_trabajo'].str.replace('Á', 'A').str.replace('É', 'E').str.replace('Í', 'I').str.replace('Ó', 'O').str.replace('Ú', 'U').str.upper()
df['tipo_orden_trabajo'] = df['tipo_orden_trabajo'].replace(np.nan, 'sin_tipo_orden_trabajo', regex=True)
# Se genera una nueva columna con TIPO VENTA OT
df["tipo_venta_ot"] = np.where(df["tipo_venta"].str.contains(re.compile("REPTOS|ANT.RPTO|CUP. MPP|RENTACAR|ANT.REPA|CHATARRA|GRUA|Deduc|CAFETERÍ|MISCELÁN",re.IGNORECASE)), "NO", "SI")
#Se Reemplaza los valores del tipo venta OT
df['tipo_venta_ot'] = df['tipo_venta_ot'].replace({'SI': 'VENTAS CON OT', 'NO': 'OTRAS VENTAS'})
#Se reemplaza los valores de la columna tipo Orden de Trabajo
df['tipo_orden_trabajo'] = df['tipo_orden_trabajo'].replace({'SIN_TIPO_ORDEN_TRABAJO': 'OTRAS VENTAS'})
df['tipo_orden_trabajo'].replace('SIN_ORDEN_TRABAJO', 'MISCELANEA', inplace=True)
df['tipo_trabajo'].replace('sin_tipo_trabajo', 'MISCELANEA', inplace=True)

df.loc[(df['contiene_obd'] == 'SI') & ((df['tipo_orden_trabajo'] == 'GARANTIA') | (df['tipo_orden_trabajo'] == 'PREENTREGA')), 'tipo_orden_trabajo'] = 'MECANICA' 
#Contar OT
df['contar_ot'] = np.where(df['tipo_venta_ot'] == 'VENTAS CON OT', 1, 0)

df['vin'] = df['vin'].astype(str)

def clean_vin(vin):
    if len(vin) != 17:
        return ''
    return vin

df['vin'] = df['vin'].apply(clean_vin)
df['patente'] = df['patente'].astype(str)

def clean_patente(patente):
    if len(patente) != 7:
        return ''
    return patente

df['patente'] = df['patente'].apply(clean_patente)
df['vin_patente'] = np.where(df['vin'] == "", df['patente'], df['vin'])
# df['numero_siniestro'].replace(0, np.nan, inplace=True)

### OBTENER ANALISIS SI TUVO OBDII

In [13]:
df['contiene_obd'] = df['contiene_obd'].map({'SI': 1, 'NO': 0})

df['tuvo_obd'] = np.where(df.groupby('vin')['contiene_obd'].transform('sum') > 0, 'SI TUVO', 'NO TUVO')

df_obd_nissan = df[df['unidad_negocios'] == 'NISSAN']

df_obd_nissan.to_excel("df_obd_nissan.xlsx", index=False)

### OBTENER COLUMNA CON LA ULTIMA MANTENCION REALIZADA

In [14]:
df['ultima_mantencion'] = np.nan
df['ultima_fecha'] = np.nan

mask = df['tipo_mantenimiento'] != 0
grouped = df[mask].groupby('vin')


results = []
for name, group in grouped:
    max_date = group['fecha_venta'].idxmax()
    row = {
        'vin': name,
        'ultima_mantencion': group['tipo_mantenimiento'].max(),
        'ultima_fecha': group['fecha_venta'].max()
    }
    results.append(row)

results_df = pd.DataFrame(results)
df = df.merge(results_df, on='vin', how='left')


### CONSULTAS PARA EXTRAER AIRLIFE - TAKATA

In [15]:
df_airlife = df.copy()
df_airlife['contiene_airlife'] = np.where(df_airlife['detalle_factura'].str.contains("TAKATA|AIRBAG|AIR BAG|AIR-BAG|INFLADOR|AIR  BAG|BOLSA DE AIRE", na=False), 'SI', 'NO')
#df_airlife.to_excel("2022_airlife.xlsx", index=False)
df_filtrado = df_airlife.query("contiene_airlife == 'SI'")
df_filtrado.to_excel("UltimoEstadoSSTT_airlife.xlsx", index=False)

### ULTIMO ESTADO BBDD SSTT

In [16]:
# ACA SE REALIZA LA AGRUPACION Y LA SEGEMNTACION 

df.sort_values(["fecha_venta","numero_orden_trabajo"], ascending=[False,False], inplace=True)
df_ultimoEstado = df.groupby("vin").first().reset_index()
df_ultimoEstado = df_ultimoEstado.astype(str)
df_ultimoEstado["fecha_venta"] = pd.to_datetime(df_ultimoEstado["fecha_venta"], format="%Y-%m-%d")
#df_ultimoEstado["fecha_estado"] = pd.to_datetime(df_ultimoEstado["fecha_estado"], format="%Y-%m-%d")
df_ultimoEstado.to_excel("UltimoEstadoSSTT.xlsx", index=False)